# <center> Test Receive CAN
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" />
### <center> Instructor: Dr. Jeremy Daily

## Prerequisites
Login tip: Send a keepalive signal every 60 seconds from the connection menu of PuTTy.

This notebook must be run on a Linux machine with SocketCAN enabled.
```
sudo ip link add type vcan 
```

If you are running from PuTTy through Windows, then be sure to start the Jupyter server with the following commands on the Beagle Bone Black connected by USB:
```
jupyter notebook --ip 192.168.7.2 --no-browser
```
Once these commands are successful, there will be a tokenized URL to paste into your browser. Note: this is not a secure connection, so be sure you are behind a firewall or on a private connection.


Be sure can-utils is installed:
```
sudo apt install can-utils
```

Here is a CAN trace to examine:

https://www.engr.colostate.edu/~jdaily/J1939/files/candump_DDEC_key_off-on_engine_running_request_on.txt

Let's add the CAN trace to the home directory:

```
wget https://www.engr.colostate.edu/~jdaily/J1939/files/candump_DDEC_key_off-on_engine_running_request_on.txt
```

Add a virtual can interface:

```
sudo ip link add dev vcan1 type vcan
sudo ip link set vcan1 up
```

Set canplayer to play an infinite loop of the CAN log in the background. Once this is going, you can logout of the machine and the vcan1 interface will still have data.

```
canplayer -l i -I candump_DDEC_key_off-on_engine_running_request_on.txt vcan1=can1 &
```

Ideally, the canplayer command will be run from a screen that can be detached from an SSH session. 

In [1]:
# Check to see if there is a file to replay CAN traffic from
# 
import os
if not os.path.exists('candump_kw_drive.txt'):
    try:
        os.system('wget https://www.engr.colostate.edu/~jdaily/J1939/files/candump_kw_drive.zip')
        os.system('unzip candump_kw_drive.zip')
        print("Downloaded and unzipped file")
    except Exception as e:
        print("There was an exeception: ", e)
else:
    print("file already exists.")

file already exists.


In [2]:
# Build a little checker to see if the canplayer is set
# Uncomment the following line if psutil is not available.
#%pip install psutil
import psutil
def check_canplayer(file):
    for p in psutil.process_iter():
        if "canplayer" in p.name():
            if file in p.cmdline():
                return True
    return False

In [4]:
# Start the canplayer if it's not running.
# This must be running on a Linux program
import subprocess
if not check_canplayer('candump_kw_drive.txt'):
    subprocess.Popen(["canplayer","-l", "i","-I","candump_kw_drive.txt","vcan0=can1"], shell=True)
    #os.system('canplayer -l i -I candump_kw_drive.txt vcan0=can1 &')
    print("started canplayer")
else: 
    print("canplayer already running")

started canplayer


## Reference Material
from https://www.kernel.org/doc/html/latest/networking/can.html#the-virtual-can-driver-vcan 
```
struct can_frame {
        canid_t can_id;  /* 32 bit CAN_ID + EFF/RTR/ERR flags */
        __u8    can_dlc; /* frame payload length in byte (0 .. 8) */
        __u8    __pad;   /* padding */
        __u8    __res0;  /* reserved / padding */
        __u8    __res1;  /* reserved / padding */
        __u8    data[8] __attribute__((aligned(8)));
};
```

Copy and modify the code from here to injest CAN data.
https://www.engr.colostate.edu/~jdaily/J1939/tools.html



In [ ]:
#!/usr/bin/python3
import socket
import struct
import time

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
interface = "vcan0"

# Bind to the interface
sock.bind((interface,))

# To match this data structure, the following struct format can be used:
can_frame_format = "<lB3x8s"

# Write date to a local log file.
with open("candump_python.log", 'w') as outfile:
    for i in range(125):
        can_packet = sock.recv(16)
        can_id, can_dlc, can_data = struct.unpack(can_frame_format, can_packet)
        extended_frame = bool(can_id & socket.CAN_EFF_FLAG)
        if extended_frame:
            can_id &= socket.CAN_EFF_MASK
            can_id_string = "{:08X}".format(can_id)
        else: #Standard Frame
            can_id &= socket.CAN_SFF_MASK
            can_id_string = "{:03X}".format(can_id)
        hex_data_string = ' '.join(["{:02X}".format(b) for b in can_data[:can_dlc]])
        print("{} {} [{}] {}".format(interface, can_id_string, can_dlc, hex_data_string))
        outfile.write("({:12.6f}) {} {}#{}\n".format(time.time(),
                                                     interface, 
                                                     can_id_string, 
                                                     hex_data_string.replace(" ","")))

The messages shown above should change based on where we are in the drive cycle. If there is no output and the indicator says In \[*], then can socket is blocking because there is no data. 